- [x] não usar categorias abaixo do nível 2. primeira alternativa) remover as categorias do dataframe; segunda alternativa) fazer checagem se a categoria esta acima do nivel 3 ou não e assim realizar o cálculo da distância.
- [x] retVal = Math.pow(rel, (1.0 - _cfg._divWeight)) * Math.pow(div, _cfg._divWeight); Ta certo, esta errado no artigo
- [ ] calculo do pr errado?? passando pelos vizinhos nao pelos log vizinhos????
- [x] score range? $score \in [0,1]?$

In [1]:
import numpy as np
import pandas as pd
import recommenders as rec
import networkx as nx
import matplotlib.pyplot as plt
import areamanager
import math

%matplotlib inline

# Category tree

In [2]:
df_categories=pd.read_json("../data/categories.json")

# dicionário alias title 2 way
dict_alias_title=dict()
for index, row in df_categories.iterrows():
    dict_alias_title[row['alias']]=row['title']
    dict_alias_title[row['title']]=row['alias']


category_tree= nx.DiGraph()
for index, row in df_categories.iterrows():
    if not row['parents']:
        category_tree.add_edge(row['alias'],'root') # root node if no parents
    else:
        for parent_label in row['parents']:
            category_tree.add_edge(row['alias'],parent_label)

undirected_category_tree=category_tree.to_undirected()
# dict alias depth
dict_alias_depth=dict()
for index, row in df_categories.iterrows():
    dict_alias_depth[row['alias']]=nx.shortest_path_length(category_tree,row['alias'],'root')

df_categories=None

# Load check-ins/reviews and treat the dataset

In [3]:
las_vegas_area=areamanager.delimiter_area('lasvegas')
def string_to_array(string):
    if string == '':
        return list()
    return eval(string)
def category_filter(categories):
    tmp_cat_list=list()
    for category in categories:
        try:
            if dict_alias_depth[dict_alias_title[category]] >= 2:
                tmp_cat_list.append(category)
        except:
            pass
#             print("Error: Category \""+category+"\" without alias name.")
    return tmp_cat_list


# read business and do some filterings
df_business=pd.read_csv("../data/business.csv",usecols=['business_id','latitude','longitude','categories'],converters={'categories':string_to_array})
df_business=areamanager.pois_in_area(las_vegas_area,df_business)
df_business.categories=df_business.categories.apply(category_filter)
# read review and merge with business
df_review=pd.read_csv("../data/review.csv",usecols=['business_id','user_id','date'])
df_review=pd.merge(df_review,df_business,on='business_id')
df_review['date']=pd.to_datetime(df_review['date'])
df_review.head()

Area selected: Las Vegas


,user_id,business_id,date,latitude,longitude,categories
0,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,2013-05-07 04:34:36,36.215546,-115.248798,"[Fitness & Instruction, Doctors, Gyms, Emergen..."
1,RBXSJA372ilErzNwz0jXvQ,ujmEBvifdJM6h6RLv4wQIg,2017-02-09 06:45:18,36.215546,-115.248798,"[Fitness & Instruction, Doctors, Gyms, Emergen..."
2,x3brMMbJrAW9PwW5A6YL5w,ujmEBvifdJM6h6RLv4wQIg,2018-08-14 05:25:33,36.215546,-115.248798,"[Fitness & Instruction, Doctors, Gyms, Emergen..."
3,Skzdl0sWhW88525a1vr59g,ujmEBvifdJM6h6RLv4wQIg,2018-05-08 19:27:59,36.215546,-115.248798,"[Fitness & Instruction, Doctors, Gyms, Emergen..."
4,3Y25VDfnQVcuc33T-U3Z6A,ujmEBvifdJM6h6RLv4wQIg,2014-10-11 06:57:18,36.215546,-115.248798,"[Fitness & Instruction, Doctors, Gyms, Emergen..."


Filtragem dos POIs que foram visitados por cinco ou mais usuários

In [4]:
df_diff_users_visited=df_review[['user_id','business_id']].drop_duplicates().reset_index(drop=True).\
groupby('business_id').count().reset_index().rename(columns={"user_id":"diffusersvisited"})

df_diff_users_visited=df_diff_users_visited[df_diff_users_visited['diffusersvisited']>=5]
del df_diff_users_visited['diffusersvisited']
df_review=pd.merge(df_review,df_diff_users_visited,on='business_id',how='inner')
df_review.count()

user_id        878996
business_id    878996
date           878996
latitude       878996
longitude      878996
categories     878996
dtype: int64

Filtragem dos usuários que realizaram vinte ou mais check-ins

In [5]:
df_review['Count']=df_review.groupby(['user_id'])['user_id'].transform('count')
df_review=df_review[df_review['Count']>=20]
del df_review['Count']
df_review.count()

user_id        194427
business_id    194427
date           194427
latitude       194427
longitude      194427
categories     194427
dtype: int64

## Add subarea information to poi

In [6]:
areamanager.poi_set_subarea(las_vegas_area,df_review,0.5).head()

,user_id,business_id,date,latitude,longitude,categories,subarea_id
9,46TNf-5T5EcNFRJPYv_nrw,ujmEBvifdJM6h6RLv4wQIg,2015-08-19 16:39:53,36.215546,-115.248798,"[Fitness & Instruction, Doctors, Gyms, Emergen...",1415.0
10,56iEnLi8jR--2ranjPSQ4w,ujmEBvifdJM6h6RLv4wQIg,2013-08-04 10:55:04,36.215546,-115.248798,"[Fitness & Instruction, Doctors, Gyms, Emergen...",1415.0
14,UDKqdVT0FrpL19shSOqgow,ujmEBvifdJM6h6RLv4wQIg,2017-03-15 23:27:38,36.215546,-115.248798,"[Fitness & Instruction, Doctors, Gyms, Emergen...",1415.0
20,Nw6Jza_cb0aqd4PT769Vzw,ujmEBvifdJM6h6RLv4wQIg,2013-01-04 22:04:02,36.215546,-115.248798,"[Fitness & Instruction, Doctors, Gyms, Emergen...",1415.0
26,3b5EsiTkqgdbxPXz7LfBpA,ujmEBvifdJM6h6RLv4wQIg,2014-08-11 23:57:54,36.215546,-115.248798,"[Fitness & Instruction, Doctors, Gyms, Emergen...",1415.0


In [7]:
# salvar aqui no arquivo .dat

# Test and training set

In [8]:
df_review=df_review.sort_values(['user_id','date']).reset_index(drop=True)

In [9]:
ranks=df_review.groupby('user_id')['date'].rank(method='first')
ranks.head()

0    1.0
1    2.0
2    3.0
3    4.0
4    5.0
Name: date, dtype: float64

In [10]:
#ranks=df_review.groupby('user_id')['date'].rank(method='first')
count=df_review['user_id'].map(df_review.groupby('user_id')['date'].apply(len))
count.head()

0    88
1    88
2    88
3    88
4    88
Name: user_id, dtype: int64

In [11]:
training_set_delimiter=(ranks/count)<0.8
test_set_delimiter=np.logical_not(training_set_delimiter)
training_set_delimiter

0          True
1          True
2          True
3          True
4          True
          ...  
194422    False
194423    False
194424    False
194425    False
194426    False
Length: 194427, dtype: bool

In [12]:
df_review_training=df_review[training_set_delimiter]
df_review_training.count()

user_id        152755
business_id    152755
date           152755
latitude       152755
longitude      152755
categories     152755
subarea_id     152755
dtype: int64

In [13]:
df_review_test=df_review[test_set_delimiter]
df_review_test.count()

user_id        41672
business_id    41672
date           41672
latitude       41672
longitude      41672
categories     41672
subarea_id     41672
dtype: int64

In [14]:
users=df_review_training['user_id'].drop_duplicates().reset_index(drop=True)
K = 80 # size of the recommendation list

In [15]:
VERY_SMALL_VALUE=-100.0

# ILD

In [16]:
def category_dis_sim(category1,category2):
    dissim=0.0
    spd=nx.shortest_path_length(undirected_category_tree,category1,category2)
    sim = 1.0 / (1.0 + spd)
    dissim=1.0-sim
    return dissim
    

def min_dist_to_list_cat(business,rec_list):
    
    min_dissim=1.0
    rec_list_size=len(rec_list)
    if rec_list_size==0:
        min_dissim=1.0
    else:
        for index,row in rec_list.iterrows():
            local_min_distance=1
            cur_distance=0
            for category1 in business['categories']:
                for category2 in row['categories']:
                    cur_distance=category_dis_sim(
                        dict_alias_title[category1],
                        dict_alias_title[category2])
                    #print(category1,category2,cur_distance,local_min_distance)
                    local_min_distance=min(local_min_distance,cur_distance)
            min_dissim=min(min_dissim,local_min_distance)
    return min_dissim
        
def objective_ild(business,rec_list):
    # /*(1-_cfg._divWeight) * relevance + */_cfg._divWeight * diversity; where _cfg._divWeight = 0.9
    ret_val= 0.0
    diversity=0.0
    diversity=min_dist_to_list_cat(business,rec_list)
    ret_val = diversity #+ (1-div_weight)*relevance
    return ret_val

# Genre Coverage

In [17]:
def relevant_categories_to_the_user(df_user_review):
    df_user_review=df_user_review.copy()
    # number of visits in pois
    df_user_review['Count']=df_user_review.groupby(['business_id','user_id'])['user_id'].transform('count')
    # merge to get mean poi visits, a column with Count and another with diffpoisvisited
    df_mean_poi_visits=pd.merge(\
    df_user_review[['user_id','Count']].groupby('user_id').sum().reset_index(level=0),\
    df_user_review[['user_id','business_id']].groupby('user_id').count().reset_index(level=0).rename(columns={"business_id":"diffpoisvisited"}),\
    on='user_id')
    
    df_mean_poi_visits['meanpoivisits']=df_mean_poi_visits['Count']/df_mean_poi_visits['diffpoisvisited']
    mean_poi_visits=df_mean_poi_visits['meanpoivisits'].iloc[0]
    # print(mean_poi_visits)
    
    # Relevant categories
    relevant_categories = set()    
    for index,row in df_user_review.iterrows():
        # Check if poi is relevant
        if row['Count'] > mean_poi_visits:
            # add relevant categories
            for category in row['categories']:
                relevant_categories.add(category)
            #print(row['business_id'])
    return relevant_categories
    #print(relevant_pois)
    

def objective_genre_coverage(poi,rec_list,df_user_review):
    relevant_categories=relevant_categories_to_the_user(df_user_review)
    count_equal=0
    categories_in_rec_list=set()
    
    for index, row in rec_list.iterrows():
        for category in row['categories']:
            categories_in_rec_list.add(category)
    
    for category in poi['categories']:
        categories_in_rec_list.add(category)
    
    #print(categories_in_rec_list)
    
    for cat1 in relevant_categories:
        for cat2 in poi['categories']:
            if cat1 == cat2:
                #print(cat1)
                count_equal=count_equal+1
    return count_equal/len(relevant_categories)


# PR/Proportionality metric

In [18]:
def haversine(lat1, lon1, lat2, lon2):
    # deg to rad
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    dlon = lon2-lon1
    dlat = lat2-lat1
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    c = 2*np.arcsin(np.sqrt(a))
    r = 6371 # km
    return c*r

def poi_distance_from_pois(poi,df_poi):
    return haversine(poi['latitude'],poi['longitude'],df_poi['latitude'],df_poi['longitude'])

def poi_neighbors(poi,df_poi,distance_km):
    return df_poi[poi_distance_from_pois(poi,df_poi)<distance_km]

def update_geo_cov(poi,df_user_review,rec_list_size,business_cover):
    user_log_size=len(df_user_review)
    neighbors=poi_neighbors(poi,df_user_review,0.1)
    num_neighbors=len(neighbors)
    vl=1
    COVER_OF_POI=user_log_size/rec_list_size
    accumulated_cover=0
    # Cover calc
    if num_neighbors<1:
        accumulated_cover=accumulated_cover+COVER_OF_POI
    else:
        cover_of_neighbor=COVER_OF_POI/num_neighbors
        for business_id in neighbors['business_id']:
            business_cover[business_id]=business_cover[business_id]+cover_of_neighbor
    accumulated_cover=accumulated_cover/user_log_size
    # end PR and DP
    DP=(accumulated_cover**2)/2
    
    for business_id in df_user_review['business_id']:
        if vl>=business_cover[business_id]:
            DP=DP+(vl-business_cover[business_id])**2    
    DP_IDEAL=user_log_size+0.5
    PR=1-DP/(DP_IDEAL)
    
    return PR




# GeoCat

In [19]:
def objective_ILD_GC_PR(poi,df_user_review,rec_list,rec_list_size,business_cover,current_proportionality,div_geo_cat_weight,div_weight):
    ild_div=objective_ild(poi,rec_list)
    
    gc_div=objective_genre_coverage(poi,rec_list,df_user_review)
    
    delta_proportionality=max(0,update_geo_cov(poi,df_user_review,rec_list_size,business_cover.copy())-current_proportionality)
    if delta_proportionality<0:
        delta_proportionality=0
    div_cat = gc_div+ild_div/rec_list_size
    div_geo = delta_proportionality
    div=div_geo_cat_weight*div_geo+(1-div_geo_cat_weight)*div_cat
    return (poi['score']**(1-div_weight))*(div**div_weight)

# GeoDiv/PR/GeoSoCa/Adaptative Kernel Estimation
Used by GeoDiv, Activity Density, Activity District

In [20]:

# def band_width_h1(df_user_poi_count,N,n):
#     sum_latitude_pois_weighted=df_user_poi_count[['count','latitude']].\
#     apply(lambda x: x['count']*x['latitude'],axis=1).sum()
    
#     sum_latitude_pois_weighted_normalized=sum_latitude_pois_weighted/N
    
#     sum_latitude_pois_weighted_squared=df_user_poi_count[['count','latitude']].apply(
#         lambda x: (x['count']*x['latitude']-sum_latitude_pois_weighted_normalized)**2,axis=1).sum()
    
#     return 1.06*(n**(-1/5))*math.sqrt(sum_latitude_pois_weighted_squared/N)

# def band_width_h2(df_user_poi_count,N,n):
#     sum_latitude_pois_weighted=df_user_poi_count[['count','longitude']].\
#     apply(lambda x: x['count']*x['longitude'],axis=1).sum()
    
#     sum_latitude_pois_weighted_normalized=sum_latitude_pois_weighted/N
    
#     sum_latitude_pois_weighted_squared=df_user_poi_count[['count','longitude']].apply(
#         lambda x: (x['count']*x['longitude']-sum_latitude_pois_weighted_normalized)**2,axis=1).sum()
    
#     return 1.06*(n**(-1/5))*math.sqrt(sum_latitude_pois_weighted_squared/N)

# def normal_kernel_kh(poi,poii,h1,h2):
#     return (1/(2*math.pi*h1*h2))*math.exp(
#         -(poi['latitude']-poii['latitude'])**2/(2*(h1**2))-(poi['longitude']-poii['longitude'])**2/(2*(h2**2))
#     )
# def kernel_density_estimation_geo(poi,df_user_poi_count,N,n,h1,h2):
#     return df_user_poi_count[['count','latitude','longitude']].apply(
#         lambda poii: poii['count']*normal_kernel_kh(poi,poii,h1,h2),axis=1).sum()/N

# def local_bandwith_hi(poi,df_user_poi_count,N,n,h1,h2,sensivity_parameter):
#     ''' sensivity_parameter \in [0,1]'''
#     geometric_mean=df_user_poi_count[['count','latitude','longitude']].apply(
#     lambda x: kernel_density_estimation_geo(x,df_user_poi_count,N,n,h1,h2),axis=1).prod()**(1/n)
#     return ((geometric_mean**(-1))*kernel_density_estimation_geo(poi,df_user_poi_count,N,n,h1,h2))**(-sensivity_parameter)

# def adaptative_kernel_kh_hi(poi,poii,df_user_poi_count,N,n,h1,h2,sensivity_parameter):
#     local_bandwith=local_bandwith_hi(poii,df_user_poi_count,N,n,h1,h2,sensivity_parameter)
#     return (1/(2*math.pi*h1*h2*(local_bandwith**2)))*math.exp(
#         -(poi['latitude']-poii['latitude'])**2/(2*((h1**2)*(local_bandwith**2)))\
#         -(poi['longitude']-poii['longitude'])**2/(2*((h2**2)*(local_bandwith**2))))

# def variables_adaptative_kernel_estimation_geo(df_user_review):
#     # N: total of checkins of user, n: total unique checkins of user
#     N=df_user_review.business_id.count()
#     n=df_user_review.business_id.nunique()
#     # dataframe with count column, count: number of visits
#     df_user_poi_count=df_user_review.copy()
#     df_user_poi_count.drop_duplicates(subset=['business_id'],inplace=True)
#     df_user_poi_count['count']=df_user_review.groupby('business_id')['user_id'].transform('count')
#     # band width calculation
#     h1=band_width_h1(df_user_poi_count,N,n)
#     h2=band_width_h2(df_user_poi_count,N,n)
#     return [df_user_poi_count,N,n,h1,h2]

# def adaptative_kernel_estimation_geo(poi,df_user_poi_count,N,n,h1,h2,sensivity_parameter):
    
#     return df_user_poi_count[['count','latitude','longitude']].apply(
#         lambda poii: poii['count']*adaptative_kernel_kh_hi(poi,poii,df_user_poi_count,N,n,h1,h2,sensivity_parameter)
#         ,axis=1).sum()/N

# SENSIVITY_PARAMETER=0.5

# Activity Density/Activity District

In [21]:
# def subarea_visit_probability(subarea_pois,df_user_poi_count,N,n,h1,h2):
#     return subarea_pois.apply(
#         lambda poi:adaptative_kernel_estimation_geo(poi,df_user_poi_count,N,n,h1,h2,SENSIVITY_PARAMETER),axis=1).sum()

# user_id=users[0]

# df_user_review=df_review_training[df_review_training['user_id']==user_id]

# vars_ake_geo=variables_adaptative_kernel_estimation_geo(df_user_review)
# vars_ake_geo[0]=vars_ake_geo[0].loc[0:6]
# # Visit probability to subareas
# subareas_visit_probability=vars_ake_geo[0].groupby('subarea_id').apply(
#     subarea_visit_probability,*vars_ake_geo).to_frame().rename(columns={0:"probability"})



In [22]:

# def user_activity_density_to_area(subarea_id,subareas_visit_probability):
#     subarea_visit_probability=subareas_visit_probability.loc[subarea_id]['probability']
#     min_visit_probability=subareas_visit_probability['probability'].max()
#     max_visit_probability=subareas_visit_probability['probability'].min()
#     return math.log(subarea_visit_probability-min_visit_probability+1)/math.log(
#     max_visit_probability-min_visit_probability+1)

# subareas_visit_probability['activity_density']=subareas_visit_probability.apply(
#     lambda subarea: user_activity_density_to_area(subarea.name,subareas_visit_probability),axis=1)
# subareas_visit_probability['activity_district']=subareas_visit_probability.apply(
#     lambda subarea: True if subarea['activity_density']>=0.9 else False,axis=1)
# subareas_visit_probability


In [28]:
K=45

for user_id in users:

    tmp_rec_list=rec.mostpopular(df_review_training,K)
    tmp_rec_list['score']=tmp_rec_list['score']/tmp_rec_list['score'].max()
    
    
    # add other columns
    tmp_rec_list=pd.merge(tmp_rec_list,df_business,on='business_id').drop_duplicates(subset=df_business.columns.difference(['categories'])).reset_index(drop=True)
    #print(tmp_rec_list)
    
    
    # User visits
    df_user_review=df_review_training[df_review_training['user_id']==user_id]
    #user_log_size=len(df_user_review)
    business_cover=dict()
    for business_id in df_user_review['business_id'].drop_duplicates().reset_index(drop=True):
        business_cover[business_id]=0
    # Constants
    #COVER_OF_POI=len(df_user_review)/len(tmp_rec_list)
    div_geo_cat_weight = 0.5 # beta,this is here because of the work to be done on parameter customization for each user
    div_weight = 0.5 # lambda
    current_proportionality=0
    
    #print(adaptative_kernel_estimation_geo(df_user_review.loc[0],df_user_review,SENSIVITY_PARAMETER))
    
    rec_list=pd.DataFrame(columns=['business_id','score','latitude','longitude','categories'])
    
    # post-processing for diversification
    for i in range(K):
        #num_current_candidates=len(tmp_rec_list)
        poi_to_insert=None
        max_objective_value=VERY_SMALL_VALUE
        for index,poi_candidate in tmp_rec_list.iterrows():
            objective_value=objective_ILD_GC_PR(poi_candidate,df_user_review,tmp_rec_list,K,\
                                                business_cover,current_proportionality,\
                                               div_geo_cat_weight,div_weight)
            
            if objective_value > max_objective_value:
                max_objective_value=objective_value
                poi_to_insert=poi_candidate
        #print(poi_to_insert)    
        
        if poi_to_insert is not None:
            poi_to_insert['score']=max_objective_value
            rec_list=rec_list.append(poi_to_insert)
            # remove from tmp_rec_list
            tmp_rec_list=tmp_rec_list[tmp_rec_list['business_id']!=poi_to_insert['business_id']].reset_index(drop=True)
            
            current_proportionality=update_geo_cov(poi_to_insert,df_user_review,K,business_cover)
        
    print(user_id)        
    print(rec_list)
    
    break

---1lKK3aKOuomHnwAkAow
               business_id     score   latitude   longitude  \
21  y8d90Pt16Nip-B5UXWBP-w  0.375291  36.156796 -115.334387   
3   KskYqH1Bi7Z_61pH6Om8pg  0.353280  36.143664 -115.141891   
23  2iTsRqUsPGRH1li1WVRvKQ  0.323054  36.167878 -115.140640   
14  ZibmYdOPKLlqDM9oR6xzOA  0.305297  36.146910 -115.300777   
5   IMLrj2klosTFvPRLv56cng  0.258668  36.160577 -115.292996   
14  rq5dgoksPHkJwJNQKlGQ7w  0.256371  36.164180 -115.289630   
14  SAIrNOB4PtDA4gziNCucwg  0.233550  36.143924 -115.191727   
15  pomGBqfbxcqPv14c3XH-ZQ  0.232379  36.298875 -115.280088   
0   g8OnV26ywJlZpezdBnOWUQ  0.229235  36.144713 -115.240092   
19  0_aeYE2-VbsZts_UpILgDw  0.218674  36.152831 -115.333867   
33  wuQDMDlqM17jQNo0lYQZ7g  0.216743  36.169326 -115.140097   
25  n8Zqqhff-2cxzWt_nwhU2Q  0.210627  36.151269 -115.332772   
25  JDZ6_yycNQFTpUZzLIKHUg  0.207145  36.134915 -115.175822   
25  Ec9CBmL3285XkeHaNp-bSQ  0.205382  36.127429 -115.224072   
4   zdE82PiD6wquvjYLyhOJNA  0.18